In [1]:
# Import MNIST data
import input_data
mnist = input_data.read_data_sets("/home/weizhuozhang/workspace/dataset/mnist", one_hot=True)

Extracting /home/weizhuozhang/workspace/dataset/mnist/train-images-idx3-ubyte.gz
Extracting /home/weizhuozhang/workspace/dataset/mnist/train-labels-idx1-ubyte.gz
Extracting /home/weizhuozhang/workspace/dataset/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/weizhuozhang/workspace/dataset/mnist/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

# Set parameters
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2

In [3]:
# TF graph input
x = tf.placeholder("float", [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder("float", [None, 10])  # 0-9 digits recognition => 10 classes

In [4]:
# Create a model

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
with tf.name_scope("Wx_b") as scope:
    # Construct a linear model
    model = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

In [6]:
# Add summary ops to collect data
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

In [7]:
# More name scopes will clean up graph representation
with tf.name_scope("cost_function") as scope:
    # Minimize error using cross entropy
    # Cross entropy
    cost_function = -tf.reduce_sum(y * tf.log(model))
    # Create a summary to monitor the cost function
    tf.summary.scalar("cost_function", cost_function)

In [8]:
with tf.name_scope("train") as scope:
    # Gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [9]:
# Initializing the variables
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [10]:
# Merge all summaries
merged_summary_op = tf.summary.merge_all()

In [11]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Set the logs write to the folder /tmp/tensorflow_logs
    summary_writer = tf.summary.FileWriter('/home/weizhuozhang/workspace/dataset/logs/', graph_def=sess.graph_def)
    
    # Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/ total_batch
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        #Display logs per iteration step
        if iteration % display_step == 0:
            print("Iteration:", '%04d'%(iteration+1), "cost=", "{:.9f}".format(avg_cost))
            
    print("Tuning completed!")
    
    # Test the model
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # Caculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Iteration: 0001 cost= 29.860461931
Iteration: 0003 cost= 21.103856619
Iteration: 0005 cost= 20.073016171
Iteration: 0007 cost= 19.652735015
Iteration: 0009 cost= 19.328712230
Iteration: 0011 cost= 19.044957861
Iteration: 0013 cost= 18.883593237
Iteration: 0015 cost= 18.737550237
Iteration: 0017 cost= 18.631951100
Iteration: 0019 cost= 18.545445969
Iteration: 0021 cost= 18.379431010
Iteration: 0023 cost= 18.249040088
Iteration: 0025 cost= 18.279057140
Iteration: 0027 cost= 18.140817799
Iteration: 0029 cost= 18.183725619
Tuning completed!
Accuracy: 0.9146
